In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow import Tensor
print("Tensorflow version is ", tf.__version__)
print('Keras version      : ',keras.__version__)
import numpy as np
import os, sys
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import h5py as h5
from sklearn.metrics import classification_report, confusion_matrix
import random
import time
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from tensorflow.keras.layers import  Dropout, Activation, GlobalAveragePooling1D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, Adadelta
from tensorflow.keras.layers import Reshape, Dense, Flatten, Add
from tensorflow.keras.activations import relu
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, History
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from random import shuffle
import glob
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle
from progressbar import ProgressBar
import matplotlib.pyplot as plt
from tensorflow.keras.utils import Sequence

Tensorflow version is  2.10.1
Keras version      :  2.10.0


In [3]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

data_file = "GOLD_XYZ_OSC.0001_1024.hdf5 "
file_handle = h5.File(data_file,'r+')

In [4]:
myData = file_handle['X'][:]  #1024x2 samples 
myMods = file_handle['Y'][:]  #mods 
mySNRs = file_handle['Z'][:]  #snrs  

print(np.shape(myData))
print(np.shape(myMods))
print(np.shape(mySNRs))
file_handle.close()

np.random.seed(0)

(2555904, 1024, 2)
(2555904, 24)
(2555904, 1)


In [5]:
snrs = list(np.unique(mySNRs.T[0]))  
print(snrs)

[-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]


In [ ]:
mods = [
    'OOK',      '4ASK',      '8ASK',      'BPSK',   'QPSK',    '8PSK',
    '16PSK',    '32PSK',     '16APSK',    '32APSK', '64APSK',  '128APSK',
    '16QAM',    '32QAM',     '64QAM',     '128QAM', '256QAM',  
    'AM-SSB-WC','AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK','OQPSK']

num_classes = np.shape(mods)[0]
print("The number of classes is ", num_classes)

In [ ]:
#  Printing one example from each modulation class

#turn off warning about more than 10 figures plotted
plt.rcParams.update({'figure.max_open_warning': 0})

def my_range(start, end, step):
    while start <= end:
        yield start
        start += step

size = np.size(myData, axis = 0)
step = size//24
# print(size)
# for x in my_range(100000, (size-1), step):
#   plt.figure()
#   plt.suptitle( mods[np.argmax(myMods[x])])
#   plt.plot(myData[x,:,0])
#   plt.plot(myData[x,:,1])

In [ ]:
# print("The number of classes is ", num_classes)
# print("myData " , np.shape(myData))
# print("myMods " , np.shape(myMods))
# print("mySNRs " , np.shape(mySNRs))
# print ("Max value of the data set = ", np.max(myData))
# print ("Min value of the data set = ", np.min(myData))
# print ("Mean value of the data set = ", np.mean(myData))
# print ("Standard Deviation of the data set ", np.std(myData) )

In [ ]:
len = (myData.shape[0])
limit = 5
Max = np.zeros(len)
Min = np.zeros(len)
for i in range(0,len):
  Max[i] = (np.max(myData[i,:,0]))
  Min[i] = (np.min(myData[i,:,0]))
  if(Max[i] > limit or Min[i] < -limit):
   print ("index =", i, mods[np.argmax(myMods[i])])
plt.figure() 
plt.plot(Max)
plt.plot(Min)

In [ ]:
#Skip this entire panel if you want to leave AM-SSB-WC and AM-SSB-SC modulations in the data set
myData = np.concatenate((myData[0:1810432], myData[2023424:2555904]),axis=0)
mySNRs = np.concatenate((mySNRs[0:1810432], mySNRs[2023424:2555904]),axis=0)
myMods = np.concatenate((myMods[0:1810432], myMods[2023424:2555904]),axis=0)

#re-onehot encode myMods to 22 from 24
length = (np.size(myMods, axis=0))
temp = np.concatenate((myMods[:,0:17],myMods[:,19:24]), axis=1)
myMods = temp

mods = [
    'OOK',      '4ASK',      '8ASK',      'BPSK',   'QPSK',    '8PSK',
    '16PSK',    '32PSK',     '16APSK',    '32APSK', '64APSK',  '128APSK',
    '16QAM',    '32QAM',     '64QAM',     '128QAM', '256QAM',  
    'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK','OQPSK']

num_classes = np.shape(mods)[0]
print("The number of classes is ", num_classes)


print(np.shape(myData))
print(np.shape(mySNRs))
print(np.shape(myMods))


print ("Max value of the data set = ", np.max(myData))
print ("Min value of the data set = ", np.min(myData))
print ("Mean value of the data set = ", np.mean(myData))
print ("Standard Deviation of the data set ", np.std(myData) )

In [ ]:
plt.figure()
plt.suptitle("SNR Distribution")
plt.hist(mySNRs, bins = [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32]) 
# plt.show()

In [ ]:
myData = myData.reshape(myData.shape[0], 1024, 1, 2) 

In [ ]:
X_train ,X_test ,Y_train ,Y_test, Z_train, Z_test =train_test_split(myData, myMods, mySNRs, test_size=0.2, random_state=0)
print (np.shape(X_test))
print (np.shape(Y_test))
print (np.shape(Z_test))
print (np.shape(X_train))
print (np.shape(Y_train))
print (np.shape(Z_train))
del myData, myMods, mySNRs

In [ ]:
input_shp = list(X_train.shape[1:])
print("Dataset Shape={0} CNN Model Input layer={1}".format(X_train.shape, input_shp))
classes = mods

In [ ]:
def resnet_block(input_data, filters, conv_size):
  x = Conv2D(filters, 1, activation=None, padding='same')(input_data)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(filters, conv_size, activation=None, padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(filters, conv_size, activation=None, padding='same')(x)
  x = BatchNormalization()(x)
  x = Add()([x, input_data])
  x = Activation('relu')(x)
    
  y = Conv2D(filters, conv_size, activation=None, padding='same')(x)
  y = BatchNormalization()(y)
  y = Activation('relu')(y)
  y = Conv2D(filters, conv_size, activation=None, padding='same')(y)
  y = BatchNormalization()(y)    
  y = Add()([y, x])
  y = Activation('relu')(y)
  
  z = MaxPooling2D(2, strides = (2,1), padding = 'same') (y)
  return z

In [ ]:
#Using 6 blocks will give small accuracy improvment with 5% decrease in performance
#num_resnet_blocks = 6

num_resnet_blocks = 6
num_filters = 32
kernel_size = 5,1

rf_input = Input(shape=input_shp, name = 'rf_input')

x = Conv2D(num_filters, (kernel_size), activation=None, padding='same')(rf_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)

for i in range(num_resnet_blocks):
    x = resnet_block(x, num_filters, (kernel_size))

x = Conv2D(num_filters, (kernel_size), activation=None, padding = 'same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# use if number of resnet blocks = 6
x = Reshape((4,4,num_filters), input_shape = (16,1,num_filters)) (x)

# use if number of resent blocks = 4
# x = Reshape((8,8,num_filters), input_shape = (32,1,num_filters)) (x)

x = GlobalAveragePooling2D()(x)
dense_1 = Dense(256, activation='relu')(x)
dropout_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(128, activation='relu')(dropout_1)
dropout_2 = Dropout(0.5)(dense_2)
dense_3 = Dense(num_classes)(dropout_2)          
softmax = Activation('softmax', name = 'softmax')(dense_3)

optimizer= Adam(learning_rate=0.00050)
model = keras.Model(rf_input, softmax)
model.compile(loss='categorical_crossentropy', metrics=["accuracy"])

print(model.summary())

In [ ]:
#  CODE FOR CLDNN ( TO USE THIS REMOVE CNN CODE GIVEN ABOVE AND ALSO DONT USE DATA GENERATOR FUNCTIN BELOW TO LOAD THE DATASET FROM CPU TO GPU)

# from keras.layers import Input, ConvLSTM2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dense, Dropout, Softmax, Reshape
# from keras.models import Model
# from keras.optimizers import Adam

# num_filters = 32
# kernel_size = (5, 1)

# rf_input = Input(shape=input_shp, name='rf_input')

# # reshape input to add the time dimension
# x = Reshape((-1, input_shp[0], input_shp[1], 1))(rf_input)

# x = ConvLSTM2D(filters=num_filters, kernel_size=kernel_size, padding='same', return_sequences=True)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)

# x = ConvLSTM2D(filters=num_filters, kernel_size=kernel_size, padding='same', return_sequences=True)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)

# x = ConvLSTM2D(filters=num_filters, kernel_size=kernel_size, padding='same', return_sequences=True)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)

# x = ConvLSTM2D(filters=num_filters, kernel_size=kernel_size, padding='same', return_sequences=False)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)

# x = GlobalAveragePooling2D()(x)
# dense_1 = Dense(256, activation='relu')(x)
# dropout_1 = Dropout(0.5)(dense_1)
# dense_2 = Dense(128, activation='relu')(dropout_1)
# dropout_2 = Dropout(0.5)(dense_2)
# dense_3 = Dense(num_classes)(dropout_2)          
# softmax = Softmax(name='softmax')(dense_3)

# optimizer = Adam(learning_rate=0.00050)
# model = Model(rf_input, softmax)
# model.compile(loss='categorical_crossentropy', metrics=["accuracy"])

# print(model.summary())


In [ ]:

predict = model.predict(X_test[0:1])
print(predict)

In [ ]:
nb_epoch = 2   # number of epochs to train on
batch_size = 1024  # training batch size


In [ ]:
### Callback
checkpoint_dir = 'resnet_checkpoints'
#os.mkdir(checkpoint_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath =checkpoint_dir + '/best_checkpoint.h5', 
                                         
                                                 verbose = 1,
                                                 save_best_only=True, 
                                                 save_weights_only=False,
                                                 mode='auto')
earlystopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        patience = 5,
        mode='auto',
        verbose = 1)

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        # return int(np.ceil(len(self.x) / float(self.batch_size)))
        return int(np.ceil(X_train.shape[0] / float(self.batch_size)))
        

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [ ]:
train_gen = DataGenerator(X_train, Y_train, 1024)
test_gen = DataGenerator(X_test, Y_test, 1024)

In [ ]:
history = model.fit(train_gen,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=1,
    validation_data=test_gen,
    callbacks = [ cp_callback,earlystopping_callback]
    )

In [ ]:
# FOR CLDNN MODEL
# history = model.fit(X_train[-30000:0],
#     Y_train[-30000:0],
#     batch_size=128,
#     epochs=nb_epoch,
#     verbose=1,
#     validation_data=(X_test, Y_test),
#     callbacks = [ cp_callback,earlystopping_callback]
#     )

In [ ]:
best_checkpoint = checkpoint_dir + '/best_checkpoint.h5'
model.load_weights(best_checkpoint)
!mkdir -p fp_model
model.save ('fp_model/resnet_fp_model.h5')

In [ ]:
# Show simple version of performance
score = model.evaluate(X_test, Y_test,  verbose=0, batch_size=batch_size)
print(score)

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.figure(figsize = (15,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    label_len = np.shape(labels)[0]
    tick_marks = np.arange(label_len)
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Plot confusion matrix
test_Y_hat = model.predict(X_test, batch_size=batch_size)
conf = np.zeros([num_classes,num_classes])
confnorm = np.zeros([num_classes,num_classes])
for i in range(0,X_test.shape[0]):
    j = list(Y_test[i,:]).index(1)
    k = int(np.argmax(test_Y_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,num_classes):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

In [ ]:
plot_confusion_matrix(confnorm, labels=classes)

In [ ]:
Y_pred = model.predict(X_test,batch_size=batch_size)
y_pred = np.argmax(Y_pred, axis = 1)
y_actual = np.argmax(Y_test, axis = 1)
classificationreport_fp = classification_report(y_actual,y_pred, target_names=mods)
print(classificationreport_fp)


In [ ]:
batchsize = 128
progress = ProgressBar()
snrlist = np.unique(Z_test)
acc_snr_arr = []

# interate over SNRs
for snr in progress(snrlist):
    acc_arr = []
    i_SNR = np.where(Z_test==snr)
    X_SNR = X_test[i_SNR[0],:,:]
    Y_SNR = Y_test[i_SNR[0],:]
    X_SNR_len = np.shape(X_SNR)[0]
    total_batches = int(X_SNR_len/batchsize)
    
    for i in (range(0, total_batches)):
        x_batch, y_batch = X_SNR[i*batchsize:i*batchsize+batchsize], Y_SNR[i*batchsize:i*batchsize+batchsize]
        
        # model prediction
        pred = model.predict(x_batch)
        
        #Pediction values are onehote, corresponding to indices representing different modulation types
        pred_ind = np.argmax(pred, axis=1)
        expected_ind = np.argmax(y_batch, axis=1)
        matches  = sum(np.equal(pred_ind, expected_ind))
        acc      = matches/batchsize
        acc_arr.append(acc)

    # Average the per-batch accuracy values
    accuracy = np.mean(acc_arr)
    acc_snr_arr.append(accuracy)
    print("SNR: ", snr, "accuracy", accuracy)

In [ ]:
plt.figure(figsize=(1,1))
plt.show()
fig= plt.figure(figsize=(10,8))
plt.plot(snrlist, acc_snr_arr, 'bo-', label='accuracy')
plt.ylabel('Accuracy')
plt.xlabel('SNR')
plt.title("Accuracy vs, SNR for Floating Point Model")
plt.legend()
plt.axis([-22, 32, 0, 1.0])
plt.grid()